Задание: загрузка результатов в БД

Соберите данные о моделях холодильников Саратов с маркетплейса beru.ru: URL, название, цена, размеры, общий объем, объем холодильной камеры.

Создайте соответствующие таблицы в SQLite базе данных и загрузите полученные данные в таблицу beru_goods.
Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:
video.ittensive.com/data/018-python-advanced/beru.ru/

In [1]:
import sqlite3  # библиотека для работы с базами данных
import requests
from bs4 import BeautifulSoup
import pandas as pd 

# Соберите данные о моделях холодильников Саратов с маркетплейса beru.ru: URL
r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/")
html = BeautifulSoup(r.content)
#print(html)
links_holod = html.find_all("a", {"class":"_3ioN70chUh Usp3kX1MNT _3Uc73lzxcf"})  # соберем все ссылки на холодильники
#print(html_holod)
links_saratov = [] # по мимо Саратова туда затесались другие, отберем только Саратов
for link in links_holod:
    if str(link).find("Саратов") > -1:
        links_saratov.append(link["href"])
#print(links_saratov)    

In [2]:
# создаем таблицу beru_goods в базе 
conn = sqlite3.connect("/SQLite/data.db3")  # конект к ранее сзданой БД
db = conn.cursor()
# db.execute("""DROP TABLE beru_goods""")
# conn.commit()
# Создадим таблицу beru_goods при помощи db.execute()
db.execute("""CREATE TABLE beru_goods
                (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
                url text,
                name text default '',
                price INTEGER default 0,
                sizes text default '',
                total_volume INTEGER default 0,
                volume_ref_camb INTEGER default 0)""")
# для того чтобы запрос выполнинся нужно выполнить conn.commit()
conn.commit()

In [3]:
# соберем информацию для загрузки в БД 
for link in  links_saratov:
    r = requests.get("https://video.ittensive.com/data/018-python-advanced/beru.ru/" + link)
    html = BeautifulSoup(r.content)
    volume = html.find_all("span", {"class":"_112Tad-7AP"})  # ищем на странице информацию по span и классу _112Tad-7AP - где указан объем 
    name = html.find("h1", {"class":"_3TfWusA7bt _26mXJDBxtH"})  # найдем имя
    price = html.find("span", {"class":"_1eyJD_sk8K"}) # найдем цену
    size = volume[0].get_text() # найдем размеры
    # теперь предстоит найти инфорамцию по обьемам камер, для этого нужно будет перебрать данные по классу _112Tad-7AP  и получить нужные
    total_volume = 0
    volume_ref_camb = 0
    for x in range(0, len(volume)):
        if volume[x].get_text().find("общий") > -1:  # ищем по совпадению общий (общий объем)
            total_volume = int(''.join(i for i in volume[x].get_text() if i.isdigit()))  # через генератор получаем только цифры объема
        elif volume[x].get_text().find("холодильной") > -1: # ищем по совпадению холодильной (объем холодильной)
            volume_ref_camb = int(''.join(i for i in volume[x].get_text() if i.isdigit()))  # через генератор получаем только цифры объема
    name = name.get_text()  # получим имя извлекая текст из html кода 
    price = int(''.join(i for i in price.get_text() if i.isdigit())) # через генератор получаем только цифры цены (без р.)
    # теперь загрузим полученные данные в базу
    db.execute("INSERT INTO  beru_goods  (url, name, price, sizes, total_volume, volume_ref_camb) VALUES (?,?,?,?,?,?)",
              (link, name, price, size, total_volume, volume_ref_camb))
    conn.commit()  # отправляем для выполнения запроса



In [4]:
import pandas as pd 
data = pd.read_sql_query("SELECT * FROM beru_goods", conn)  # Первый параметр это запрос, второй это коннект к БД
print(data)
db.close()  # закрываем БД

   id                                                url  \
0   1               kholodilnik-saratov-452-ksh-120.html   
1   2           kholodilnik-saratov-263-kshd-200-30.html   
2   3           kholodilnik-saratov-209-kshd-275-65.html   
3   4  kholodilnik-saratov-209-belyi-s-chernymi-nakla...   
4   5         kholodilnik-saratov-452-ksh-120-seryi.html   
5   6       kholodilnik-saratov-550-ksh-120-bez-nto.html   
6   7               kholodilnik-saratov-467-ksh-210.html   
7   8           kholodilnik-saratov-264-kshd-150-30.html   
8   9     kholodilnik-saratov-264-kshd-150-30-seryi.html   
9  10                 kholodilnik-saratov-263-seryi.html   

                                                name  price  \
0                   Холодильник Саратов 452 (КШ-120)  10728   
1               Холодильник Саратов 263 (КШД-200/30)  14299   
2               Холодильник Саратов 209 (КШД 275/65)  13292   
3  Холодильник Саратов 209 белый с черными наклад...  18259   
4             Холодильни